In [1]:
import numpy as np
import pandas as pd
import sys 
import os

# Setup paths
PROJECT_ROOT = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 'filter_sparsity')
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)
    
print(PROJECT_ROOT)

from polar_ns.train import fit_model
from polar_ns.prune import main
from polar_ns.fine_tune import fine_tune_model

c:\Users\laeti\SHK_NODE\filter_sparsity


In [9]:
# For NS: use lbd/s/sr as lambda value
# %% crreate lambda space
lambda_min = 1e-4
#lambda_min = 0.0014
lambda_max = 5
lambda_seq_len = 10
lambda_seq = np.exp(np.linspace(np.log(lambda_max), np.log(lambda_min), lambda_seq_len))
lambda_seq = np.concatenate([lambda_seq, [0]])

In [10]:
lambda_seq

array([5.00000000e+00, 1.50266519e+00, 4.51600535e-01, 1.35720881e-01,
       4.07886087e-02, 1.22583245e-02, 3.68403150e-03, 1.10717318e-03,
       3.32742119e-04, 1.00000000e-04, 0.00000000e+00])

In [15]:
# config for pp via loss type = "sr"
def load_config_train(lbd, path_sv, path_bckp, path_log):
    config_train = {
    #'config' : [6, 'A', 16, 'A'],
    'loss' : 'sr', 
    'lbd' : lbd, 
    #'alpha' : 1, 
    #'t' : 1,
    'epochs' : 50, 
    'batch_size' : 256, 
    'test_batch_size' : 256,
    'max_epoch' : None, 
    'lr' : 0.15, 
    'momentum' : 0.9, 
    'weight_decay': 0.0, 
    'resume' : None,
    'no_cuda': True, 
    'seed' : 1234, 
    'log_interval' : 10,
    'bn_init_value' : 0.5, 
    # only used for pp
    'clamp' : 1.0, 
    'gate' : False, 
    'flops_weighted' : False,
    'weight-max': None, 
    'weight-min' : None, 
    'bn_wd' : True, 
    'target-flops' : None, 
    'debug' : False,
    'arch' : 'leNet', 
    'retrain' : False, 
    'save' : path_sv, 
    'backup' : path_bckp, 
    'log' : path_log
    #'save' : './checkpoints_ns/', 
    #'backup' : './backup_ns/', 
    #'log' : './events_ns/'
    }
    
    return config_train

In [12]:
def load_config_prune(path_model, path_save):
    config_prune = {
    'model' : path_model,
    'batch_size': 256, 
    'test_batch_size' : 256,
    'no_cuda': True, 
    # Type of pruning/where to prune on; 'polarization', 'l1-norm', 'ns'
    'prune_type': 'ns', 
    # How to find prunish threshold; only applied if prune_type == ploarization; ["fixed", "grad", "search"]; for og. PP: grad
    'pruning_strategy' : None,
    #'Pruning ratio of the L1-Norm'; only applied if prune_type == l1-norm OR ns, default none --> ratio to be pruned
    'l1_norm_ratio': 0.1,  
    # l1_norm_cutoff for other ns and pp threshold finding --> IMPLEMENTED ; CAN BE USED!
    'l1_norm_cutoff': None,
    # None` or `"default"`: default behaviour. The pruning threshold is determined by `sparse_layer
    'prune_mode' : 'default', 
    'save' : './checkpoints_ns/',
    'gate' : False
    }
    return config_prune

In [13]:
def load_config_finetune(path_sv, path_bckp, path_log, path_refine):
    config_finetune = {
    #'config' : [6, 'A', 16, 'A'],
    'cuda' : False, 
    'no_cuda' : True, 
    # only relevant if sr TRUE, sr = sparsity regularization --> same as lbd in main
    's' : 0.0001, 
    'sr' : False, # no further sr 
    'epochs' : 50, 
    'batch_size' : 256, 
    'test_batch_size' : 256,
    'max_epoch' : None, 
    'lr' : 0.15, 
    'momentum' : 0.9, 
    'weight_decay': 0.0, 
    'seed' : 1234, 
    'log_interval' : 10,
    'gate' : False, 
    'flops_weighted' : False,
    'bn_wd' : True, 
    'resume' : None,
    'arch' : 'leNet', 
    #'refine' : './checkpoints/pruned_grad.pth.tar',
    'refine': path_refine,
    'save' : path_sv,
    'backup' : path_bckp,
    'log' : path_log}
    return config_finetune

In [14]:
def generate_paths(lbd): 
    save =  './checkpoints_ns_' + str(lbd) +'/'
    backup = './backup_ns_'+ str(lbd) +'/'
    log = './events_ns_/' + str(lbd) + '/'
    return save, backup, log

In [8]:
for lbd in lambda_seq:
    print(lbd)
    path_sv, path_bckp, path_log = generate_paths(lbd)
    path_model = path_sv + 'model_best.pth.tar',
    path_refine =  path_sv + 'pruned_grad.pth.tar',
    
    cfg_train =  load_config_train(lbd, path_sv, path_bckp, path_log)
    fit_model(cfg_train)
    
    cfg_prune = load_config_prune(path_model, path_sv)
    main(cfg_prune)
    cfg_finetune = load_config_finetune(path_sv, path_bckp, path_log, path_refine)
    fine_tune_model(cfg_finetune)

0.010000000000000004
LeNet5 make_layers: feature cfg [6, 'A', 16, 'A']
Weight decay param: parameter name feature.0.conv.weight
Weight decay param: parameter name feature.0.batch_norm.weight
Weight decay param: parameter name feature.0.batch_norm.bias
Weight decay param: parameter name feature.2.conv.weight
Weight decay param: parameter name feature.2.batch_norm.weight
Weight decay param: parameter name feature.2.batch_norm.bias
Weight decay param: parameter name classifier.0.weight
Weight decay param: parameter name classifier.0.bias
Weight decay param: parameter name classifier.2.weight
Weight decay param: parameter name classifier.2.bias
Weight decay param: parameter name classifier.4.weight
Weight decay param: parameter name classifier.4.bias
Start epoch 0/50...
Step: 1 Train Epoch: 0 [0/60000 (0.0%)]	Loss: 2.302664
Step: 11 Train Epoch: 0 [2560/60000 (4.3%)]	Loss: 2.300104
Step: 21 Train Epoch: 0 [5120/60000 (8.5%)]	Loss: 2.302358
Step: 31 Train Epoch: 0 [7680/60000 (12.8%)]	Loss:

c:\Users\laeti\SHK_NODE\filter_sparsity\.venv\Lib\site-packages\torch\nn\_reduction.py:51: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 2.3034, Accuracy: 1000/10000 (10.0000%)

Start epoch 1/50...
Step: 236 Train Epoch: 1 [0/60000 (0.0%)]	Loss: 2.300089
Step: 246 Train Epoch: 1 [2560/60000 (4.3%)]	Loss: 2.300534
Step: 256 Train Epoch: 1 [5120/60000 (8.5%)]	Loss: 2.301837
Step: 266 Train Epoch: 1 [7680/60000 (12.8%)]	Loss: 2.304384
Step: 276 Train Epoch: 1 [10240/60000 (17.0%)]	Loss: 2.302490
Step: 286 Train Epoch: 1 [12800/60000 (21.3%)]	Loss: 2.304910
Step: 296 Train Epoch: 1 [15360/60000 (25.5%)]	Loss: 2.302492
Step: 306 Train Epoch: 1 [17920/60000 (29.8%)]	Loss: 2.304504
Step: 316 Train Epoch: 1 [20480/60000 (34.0%)]	Loss: 2.302083
Step: 326 Train Epoch: 1 [23040/60000 (38.3%)]	Loss: 2.301140
Step: 336 Train Epoch: 1 [25600/60000 (42.6%)]	Loss: 2.302963
Step: 346 Train Epoch: 1 [28160/60000 (46.8%)]	Loss: 2.303276
Step: 356 Train Epoch: 1 [30720/60000 (51.1%)]	Loss: 2.303049
Step: 366 Train Epoch: 1 [33280/60000 (55.3%)]	Loss: 2.303684
Step: 376 Train Epoch: 1 [35840/60000 (59.6%)]	Loss: 2.3

TypeError: Module.load_state_dict() missing 1 required positional argument: 'state_dict'

In [ ]:
# def repr_and_saves(seed, cuda = False, log = 'logs', save = 'results', backup_path = 'backups'):
#     torch.manual_seed(seed)
#     np.random.seed(seed)
#     if cuda:
#         torch.cuda.manual_seed(seed)
#         torch.backends.cudnn.deterministic = True
#         torch.backends.cudnn.benchmark = False

#     if not os.path.exists(save):
#         os.makedirs(save)
#     if backup_path is not None and not os.path.exists(backup_path):
#         os.makedirs(backup_path)
#     if not os.path.exists(log):
#         os.makedirs(log)

# def get_loaders(batch_size, test_batch_size): 
#     train_loader = torch.utils.data.DataLoader(
#     datasets.FashionMNIST('./data.fashionMNIST', train=True, download=True,
#                     transform=transforms.Compose([
#                         transforms.Pad(2),
#                         #transforms.RandomCrop(32),
#                         #transforms.RandomHorizontalFlip(),
#                         transforms.ToTensor(),
#                         transforms.Normalize((0.5,), (0.5,))
#                     ])),
#     batch_size=batch_size, shuffle=True)

#     test_loader = torch.utils.data.DataLoader(
#     datasets.FashionMNIST('./data.fashionMNIST', train=False, transform=transforms.Compose([
#         transforms.Pad(2),
#         transforms.ToTensor(),
#         transforms.Normalize((0.5,), (0.5,))
#     ])),
#     batch_size=test_batch_size, shuffle=True)
    
#     return train_loader, test_loader


# def freeze_sparse_gate(model: nn.Module):
#     # do not update all SparseGate
#     for sub_module in model.modules():
#         if isinstance(sub_module, models.common.SparseGate):
#             for p in sub_module.parameters():
#                 # do not update SparseGate
#                 p.requires_grad = False
                
# def define_optim(model, bn_wd, lr, momentum, weight_decay): 
#     if bn_wd:
#         no_wd_type = [models.common.SparseGate]
#     else:
#         # do not apply weight decay on bn layers
#         no_wd_type = [models.common.SparseGate, nn.BatchNorm2d, nn.BatchNorm1d]

#     no_wd_params = []  # do not apply weight decay on these parameters
#     for module_name, sub_module in model.named_modules():
#         for t in no_wd_type:
#             if isinstance(sub_module, t):
#                 for param_name, param in sub_module.named_parameters():
#                     no_wd_params.append(param)
#                     print(f"No weight decay param: module {module_name} param {param_name}")

#     no_wd_params_set = set(no_wd_params)  # apply weight decay on the rest of parameters
#     wd_params = []
#     for param_name, model_p in model.named_parameters():
#         if model_p not in no_wd_params_set:
#             wd_params.append(model_p)
#             print(f"Weight decay param: parameter name {param_name}")

#     optimizer = torch.optim.SGD([{'params': list(no_wd_params), 'weight_decay': 0.},
#                                 {'params': list(wd_params), 'weight_decay': weight_decay}],
#                                 lr,
#                                 momentum=momentum)
    
#     return optimizer



# def bn_weights(model):
#     weights = []
#     bias = []
#     for name, m in model.named_modules():
#         if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
#             weights.append((name, m.weight.data))
#             bias.append((name, m.bias.data))

#     return weights, bias
#     pass


# # def adjust_learning_rate(optimizer, epoch, gammas, schedule, config):
# #     """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
# #     lr = config.get('lr')
# #     assert len(gammas) == len(schedule), "length of gammas and schedule should be equal"
# #     for (gamma, step) in zip(gammas, schedule):
# #         if epoch >= step:
# #             lr = lr * gamma
# #         else:
# #             break
# #     for param_group in optimizer.param_groups:
# #         param_group['lr'] = lr
# #     return lr


# # additional subgradient descent on the sparsity-induced penalty term
# def updateBN(config, model):
#     if config.get('loss') == LossType.L1_SPARSITY_REGULARIZATION:
#         sparsity = config.get('lbd')
#         bn_modules = list(filter(lambda m: (isinstance(m[1], nn.BatchNorm2d) or isinstance(m[1], nn.BatchNorm1d)),
#                                  model.named_modules()))
#         bn_modules = list(map(lambda m: m[1], bn_modules))  # remove module name
#         for m in bn_modules:
#             if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
#                 m.weight.grad.data.add_(sparsity * torch.sign(m.weight.data))
#     else:
#         raise NotImplementedError(f"Do not support loss: {config.get('loss')}")


# def clamp_bn(model, lower_bound=0, upper_bound=1):
#     if model.gate:
#         sparse_modules = list(filter(lambda m: isinstance(m, SparseGate), model.modules()))
#     else:
#         sparse_modules = list(
#             filter(lambda m: isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d), model.modules()))

#     for m in sparse_modules:
#         m.weight.data.clamp_(lower_bound, upper_bound)


# def set_bn_zero(model: nn.Module, threshold=0.0):
#     """
#     Set bn bias to zero
#     Note: The operation is inplace. Parameters of the model will be changed!
#     :param model: to set
#     :param threshold: set bn bias to zero if corresponding lambda <= threshold
#     :return modified model, the number of zero bn channels
#     """
#     with torch.no_grad():
#         mask_length = 0
#         for name, sub_module in model.named_modules():
#             # only process bn modules
#             if not (isinstance(sub_module, nn.BatchNorm1d) or isinstance(sub_module, nn.BatchNorm2d)):
#                 continue

#             mask = sub_module.weight.detach() <= threshold
#             sub_module.weight[mask] = 0.
#             sub_module.bias[mask] = 0.

#             mask_length += torch.sum(mask).item()

#     return model, mask_length


# def bn_sparsity(model, loss_type, sparsity, t, alpha,
#                 flops_weighted: bool, weight_min=None, weight_max=None):
#     """

#     :type model: torch.nn.Module
#     :type alpha: float
#     :type t: float
#     :type sparsity: float
#     :type loss_type: LossType
#     """
#     bn_modules = model.get_sparse_layers()

#     if loss_type == LossType.POLARIZATION or loss_type == LossType.L2_POLARIZATION:
#         # compute global mean of all sparse vectors
#         n_ = sum(map(lambda m: m.weight.data.shape[0], bn_modules))
#         sparse_weights_mean = torch.sum(torch.stack(list(map(lambda m: torch.sum(m.weight), bn_modules)))) / n_

#         sparsity_loss = 0.
#         if flops_weighted:
#             for sub_module in model.modules():
#                 if isinstance(sub_module, model.building_block):
#                     flops_weight = sub_module.get_conv_flops_weight(update=True, scaling=True)
#                     sub_module_sparse_layers = sub_module.get_sparse_modules()

#                     for sparse_m, flops_w in zip(sub_module_sparse_layers, flops_weight):
#                         # linear rescale the weight from [0, 1] to [lambda_min, lambda_max]
#                         flops_w = weight_min + (weight_max - weight_min) * flops_w

#                         sparsity_term = t * torch.sum(torch.abs(sparse_m.weight.view(-1))) - torch.sum(
#                             torch.abs(sparse_m.weight.view(-1) - alpha * sparse_weights_mean))
#                         sparsity_loss += flops_w * sparsity * sparsity_term
#             return sparsity_loss
#         else:
#             for m in bn_modules:
#                 if loss_type == LossType.POLARIZATION:
#                     sparsity_term = t * torch.sum(torch.abs(m.weight)) - torch.sum(
#                         torch.abs(m.weight - alpha * sparse_weights_mean))
#                 elif loss_type == LossType.L2_POLARIZATION:
#                     sparsity_term = t * torch.sum(torch.abs(m.weight)) - torch.sum(
#                         (m.weight - alpha * sparse_weights_mean) ** 2)
#                 else:
#                     raise ValueError(f"Unexpected loss type: {loss_type}")
#                 sparsity_loss += sparsity * sparsity_term

#             return sparsity_loss
#     else:
#         raise ValueError()


# def train(model, epoch, train_loader, optimizer, lr_scheduler, config, history_score, global_step):
#     model.train()
#     #global history_score, global_step 
#     avg_loss = 0.
#     avg_sparsity_loss = 0.
#     train_acc = 0.
#     total_data = 0
#     for batch_idx, (data, target) in enumerate(train_loader):
#         if config.get('cuda'):
#             data, target = data.cuda(), target.cuda()
#         optimizer.zero_grad()
#         output = model(data)
#         if isinstance(output, tuple):
#             output, output_aux = output
#         loss = F.cross_entropy(output, target)

#         # logging
#         avg_loss += loss.data.item()
#         pred = output.data.max(1, keepdim=True)[1]
#         train_acc += pred.eq(target.data.view_as(pred)).cpu().sum()
#         total_data += target.data.shape[0]

#         if config.get('loss') in {LossType.POLARIZATION,
#                          LossType.L2_POLARIZATION}:
#             sparsity_loss = bn_sparsity(model, config.get('loss'), config.get('lbd'),
#                                         t=config.get('t'), alpha=config.get('alpha'),
#                                         flops_weighted=config.get('flops_weighted'),
#                                         weight_max=config.get('weight_max'), weight_min=config.get('weight_min'))
#             loss += sparsity_loss
#             avg_sparsity_loss += sparsity_loss.data.item()
#         loss.backward()
        
#         if config.get('loss') in {LossType.L1_SPARSITY_REGULARIZATION}:
#             updateBN(config, model)
            
#         optimizer.step()
#         if config.get('loss') in {LossType.POLARIZATION,
#                          LossType.L2_POLARIZATION}:
#             clamp_bn(model, upper_bound=config.get('clamp'))
#         global_step += 1
        
#         lr_scheduler.step()
        
#         if batch_idx % config.get('log_interval') == 0:
#             print('Step: {} Train Epoch: {} [{}/{} ({:.1f}%)]\tLoss: {:.6f}'.format(
#                 global_step, epoch, batch_idx * len(data), len(train_loader.dataset),
#                                     100. * batch_idx / len(train_loader), loss.data.item()))

#     history_score[epoch][0] = avg_loss / len(train_loader)
#     history_score[epoch][1] = float(train_acc) / float(total_data)
#     history_score[epoch][3] = avg_sparsity_loss / len(train_loader)
#     return history_score, global_step


# def test(model, test_loader, config):
#     model.eval()
#     test_loss = 0
#     correct = 0
#     with torch.no_grad():
#         for data, target in test_loader:
#             if config.get('cuda'):
#                 data, target = data.cuda(), target.cuda()
#             output = model(data)
#             if isinstance(output, tuple):
#                 output, output_aux = output
#             test_loss += F.cross_entropy(output, target, size_average=False).data.item()  # sum up batch loss
#             pred = output.data.max(1, keepdim=True)[1]  # get the index of the max log-probability
#             correct += pred.eq(target.data.view_as(pred)).cpu().sum()

#     test_loss /= len(test_loader.dataset)
#     print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
#         test_loss, correct, len(test_loader.dataset),
#         100. * float(correct) / len(test_loader.dataset)))
#     return float(correct) / float(len(test_loader.dataset))


# def save_checkpoint(state, is_best, filepath, backup: bool, backup_path: str, epoch: int, max_backup: int, config):
#     state['config'] = config

#     torch.save(state, os.path.join(filepath, 'checkpoint.pth.tar'))
#     if is_best:
#         shutil.copyfile(os.path.join(filepath, 'checkpoint.pth.tar'), os.path.join(filepath, 'model_best.pth.tar'))
#     if backup and backup_path is not None:
#         shutil.copyfile(os.path.join(filepath, 'checkpoint.pth.tar'),
#                         os.path.join(backup_path, 'checkpoint_{}.pth.tar'.format(epoch)))

#         if max_backup is not None:
#             while True:
#                 # remove redundant backup checkpoints to save space
#                 checkpoint_match = map(lambda f_name: re.fullmatch("checkpoint_([0-9]+).pth.tar", f_name),
#                                        os.listdir(backup_path))
#                 checkpoint_match = filter(lambda m: m is not None, checkpoint_match)
#                 checkpoint_id: typing.List[int] = list(map(lambda m: int(m.group(1)), checkpoint_match))
#                 checkpoint_count = len(checkpoint_id)
#                 if checkpoint_count > max_backup:
#                     min_checkpoint_epoch = min(checkpoint_id)
#                     min_checkpoint_path = os.path.join(backup_path,
#                                                        'checkpoint_{}.pth.tar'.format(min_checkpoint_epoch))
#                     print(f"Too much checkpoints (Max {max_backup}, got {checkpoint_count}).")
#                     print(f"Remove file: {min_checkpoint_path}")
#                     os.remove(min_checkpoint_path)
#                 else:
#                     break


In [ ]:
# def fit_model(config): 
#     config['cuda'] = not config.get('no_cuda') and torch.cuda.is_available()
#     config['loss'] = LossType.from_string(config.get('loss'))
#     #config.get('decay_epoch') = sorted([int(config.get('epochs') * i if i < 1 else i) for i in config.get('decay_epoch')])
#     if not config.get('seed'):
#         config['seed'] = random.randint(500, 1000)
    
#     repr_and_saves(config.get('seed'))

#     train_loader, test_loader = get_loaders(config.get('batch_size'), config.get('test_batch_size'))
#     num_classes = 10 
    
    
#     if not config.get('retrain'):
#         model = LeNet5(gate=config.get('gate'), bn_init_value=config.get('bn_init_value'))
#     else:  # initialize model for retraining with configs
#         checkpoint = torch.load(config.get('retrain'))
#         if config.get('arch') == "leNet":
#             model = models.__dict__[config.get('arch')](num_classes=num_classes, cfg=checkpoint['cfg'])
#         else:
#             raise NotImplementedError(f"Do not support {config.get('arch')} for retrain.")
        
#     if config.get('fix_gate'):
#         if config.get('lbd') != 0:
#             raise ValueError("The lambda must be 0 in fix-gate mode.")
#         # do not update all SparseGate
#         freeze_sparse_gate(model)
        
#     optimizer = define_optim(model, config.get('bn_wd'), config.get('lr'), config.get('momentum'), config.get('weight_decay'))
#     lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 100, eta_min=0.0, last_epoch=-1, verbose='deprecated')
    
#     best_prec1 = 0.0
#     global_step = 0
#     writer = SummaryWriter(logdir=config.get('log', 'logs'))
#     history_score = np.zeros((config.get('epochs'), 6))

#     for epoch in range(config.get('start_epoch', 0), config.get('epochs')):
#         if config.get('max_epoch') is not None and epoch >= config.get('max_epoch'):
#             break

#         #current_learning_rate = adjust_learning_rate(optimizer, epoch, config.get('gammas'), config.get('decay_epoch'), config)
#         print("Start epoch {}/{}...".format(epoch, config.get('epochs')))

#         #weights, bias = bn_weights(model)
        
#         weights, bias = bn_weights(model)
#         for bn_name, bn_weight in weights:
#             writer.add_histogram("bn/" + bn_name, bn_weight, global_step=epoch)
#         for bn_name, bn_bias in bias:
#             writer.add_histogram("bn_bias/" + bn_name, bn_bias, global_step=epoch)
#         # visualize conv kernels
#         for name, sub_modules in model.named_modules():
#             if isinstance(sub_modules, nn.Conv2d):
#                 writer.add_histogram("conv_kernels/" + name, sub_modules.weight, global_step=epoch)
#         if config['gate']:
#             for gate_name, m in model.named_modules():
#                 if isinstance(m, SparseGate):
#                     writer.add_histogram("gate/" + gate_name, m.weight, global_step=epoch)

        
#         history_score, global_step = train(model, epoch, train_loader, optimizer, lr_scheduler, config, history_score, global_step)

#         prec1 = test(model, test_loader, config)
#         history_score[epoch][2] = prec1
#         np.savetxt(os.path.join(config.get('save'), 'record.txt'), history_score, fmt='%10.5f', delimiter=',')
#         is_best = prec1 > best_prec1
#         best_prec1 = max(prec1, best_prec1)
#         save_checkpoint({
#             'epoch': epoch + 1,
#             'state_dict': model.state_dict(),
#             'best_prec1': best_prec1,
#             'optimizer': optimizer.state_dict(),
#         }, is_best, filepath=config.get('save', 'results'),
#             backup_path=config.get('backup_path', 'backups'),
#             backup=epoch % config.get('backup_freq', 10) == 0,
#             epoch=epoch,
#             max_backup=config.get('max_backup', 25), 
#             config = config
#         )
        
        
#         # write the tensorboard
#         writer.add_scalar("train/average_loss", history_score[epoch][0], epoch)
#         writer.add_scalar("train/sparsity_loss", history_score[epoch][3], epoch)
#         writer.add_scalar("train/train_acc", history_score[epoch][1], epoch)
#         writer.add_scalar("train/lr", optimizer.param_groups[0]['lr'], epoch)
#         writer.add_scalar("val/acc", prec1, epoch)
#         writer.add_scalar("val/best_acc", best_prec1, epoch)


#         # flops
#         # if config.get('loss') in {LossType.POLARIZATION, LossType.L2_POLARIZATION}:
#         #     flops_grad, flops_fixed, baseline_flops = prune_while_training(model, arch=config.get('arch'),
#         #                                                                 prune_mode="default",
#         #                                                                 num_classes=num_classes)
#         #     print(f" --> FLOPs in epoch (grad) {epoch}: {flops_grad:,}, ratio: {flops_grad / baseline_flops}")
#         #     print(f" --> FLOPs in epoch (fixed) {epoch}: {flops_fixed:,}, ratio: {flops_fixed / baseline_flops}")
#         #     if config.get('loss') == LossType.POLARIZATION and config.get('target_flops') and (
#         #             flops_grad / baseline_flops) <= config.get('target_flops') and config.get('gate'):
#         #         print("The grad pruning FLOPs archieve the target FLOPs.")
#         #         print(f"Current pruning ratio: {flops_grad / baseline_flops}")
#         #         print("Stop polarization from current epoch and continue training.")

#         #         # do not apply polarization loss
#         #         config['lbd'] = 0
#         #         freeze_sparse_gate(model)
#         #         if config.get('backup_freq') > 20:
#         #             config['backup_freq'] = 20

#     # if config.get('loss') == LossType.POLARIZATION and config.get('target_flops') and (
#     #         flops_grad / baseline_flops) > config.get('target_flops') and config.get('gate'):
#     #     print("WARNING: the FLOPs does not achieve the target FLOPs at the end of training.")
#     print("Best accuracy: " + str(best_prec1))
#     history_score[-1][0] = best_prec1
#     np.savetxt(os.path.join(config.get('save'), 'record.txt'), history_score, fmt='%10.5f', delimiter=',')

#     writer.close()

#     print("Best accuracy: " + str(best_prec1))    